In [17]:
import pandas as pd
import csv
import pandas.io.gbq as bq

In [18]:
Labels = '''Trawlers
Purse_seines
Drifting_longlines
Fixed_gear
Set_gillnets
Pots_and_traps
Set_longlines
longline - don't know if set or longline
Squid_jigger
Pole_and_line
Troller
other_fishing
unknown_fishing
Reefer
Cargo_Tanker
Tanker
Cargo
Passenger
Sailing
Seismic_vessel
Tug_Pilot
Tug
Pilot
other_not_fishing
unknown_but_not_fishing
no_idea_what_it_is'''.split("\n")

In [19]:
# first find the geartypes

q = '''select geartype, count(*) num from (select a.mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number) group by geartype order by 2 desc'''

eu_gear =pd.io.gbq.read_gbq(q, project_id='world-fishing-827' )

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 27 rows.

Total time taken 0.85 s.
Finished at 2017-01-12 09:28:01.


In [20]:
# see http://ec.europa.eu/fisheries/fleet/index.cfm?method=Codification.Cod_gear
eu_gear_codes = {"DRB":"Boat dredges",
"DRH":"Hand dredges operating from a boat",
"HMD":"Mechanised dredges including suction dredges",
"GNC":"Encircling gillnets",
"GND":"Drift nets",
"GNS":"Set gillnets (anchored)",
"GTN":"Combined gillnets-trammel nets",
"GTR":"Trammel nets",
"HAR":"Harpoons",
"LLS":"Set longlines",
"LLD":"Drifting longlines",
"LHM":"Handlines and pole-lines (mechanised)",
"LHP":"Handlines and pole-lines (hand operated)",
"LLD":"Drifting longlines",
"LLS":"Set longlines",
"LTL":"Trolling lines",
"LNB":"Boat-operated lift nets",
"LNS":"Shore-operated stationary lift nets",
"NO":"No gear",
"SB":"Beach seines",
"SDN":"Danish seines",
"SPR":"Pair seines",
"SSC":"Scottish seines",
"LA":"Lampara nets",
"PS":"Purse seines",
"FPO":"Pots and traps",
"OTB":"Bottom otter trawls",
"OTM":"Midwater otter trawls",
"OTT":"Otter twin trawls",
"PTB":"Bottom pair trawls",
"PTM":"Midwater pair trawls",
"TBB":"Beam trawls",
"NK":"Unknown gear"};


for index, row in eu_gear.iterrows():
    print eu_gear_codes[row['geartype']], row['num']

Bottom otter trawls 3118
Purse seines 779
Beam trawls 517
Set gillnets (anchored) 487
Midwater otter trawls 311
Set longlines 241
Boat dredges 229
Drifting longlines 206
Pots and traps 141
Trammel nets 100
Otter twin trawls 66
Mechanised dredges including suction dredges 59
Handlines and pole-lines (hand operated) 48
Bottom pair trawls 43
Danish seines 35
Midwater pair trawls 29
Scottish seines 23
Drift nets 16
Hand dredges operating from a boat 14
Combined gillnets-trammel nets 7
Trolling lines 7
Pair seines 3
Encircling gillnets 3
Boat-operated lift nets 2
Beach seines 2
No gear 1
Lampara nets 1


In [21]:
eu_gear2 = {"DRB":"Trawlers",
"DRH":"Trawlers",
"HMD":"Trawlers",
"GNC":"Set_gillnets",
"GND":"other_fishing",
"GNS":"Set_gillnets",
"GTN":"Set_gillnets",
"GTR":"Set_gillnets",
"HAR":"other_fishing",
"LHM":"Pole_and_line",
"LHP":"Pole_and_line",
"LLD":"Drifting_longlines",
"LLS":"Set_longlines",
"LTL":"Trollers",
"LNB":"other_fishing",
"LNS":"other_fishing",
"NO":"other_fishing",
"SB":"other_fishing",
"SDN":"other_fishing",
"SPR":"other_fishing",
"SSC":"other_fishing",
"LA":"Set_gillnets",
"PS":"Purse_seines",
"FPO":"Pots_and_traps",
"OTB":"Trawlers",
"OTM":"Trawlers",
"OTT":"Trawlers",
"PTB":"Trawlers",
"PTM":"Trawlers",
"TBB":"Trawlers",
"NK":"unknown_fishing",
"NO":None}

for e in eu_gear2:
    if eu_gear2[e] not in Labels:
        print e

LTL
NO


In [31]:
for e in eu_gear_codes:
    if eu_gear2[e]=="other_fishing":
        print eu_gear_codes[e]

Drift nets
Harpoons
Pair seines
Shore-operated stationary lift nets
Beach seines
Danish seines
Boat-operated lift nets
Scottish seines


In [22]:
# get EU values

q = '''
select mmsi, 
group_concat(geartype) geartype, 
group_concat(secondary_geartype) secondary_geartype,
avg(length) length, 
avg(tonnage) tonnage,
avg(power) power,
count(*) num_mmsi,
from
(select a.mmsi mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage, b.Gear_Sec_Code as secondary_geartype,
b.Power_Main as power
from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number)
group by mmsi
having num_mmsi = 1

'''
eu = bq.read_gbq(q, project_id = "world-fishing-827")

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 6439 rows.

Total time taken 1.4 s.
Finished at 2017-01-12 09:28:05.


In [23]:
list_override = {}
with open('other_list_files/double_geartype_override.csv', 'rU') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        mmsi = row['mmsi']
        geartype = row['newlabel']
        list_override[mmsi]=geartype

In [24]:
g = pd.read_csv('other_list_files/double_geartype_override.csv')
g.groupby('newlabel').count()

,mmsi,link,primary geartyp,secondary geartype,Vaida's Best Guess,David's Best Guess,Kristina's Best Guess
newlabel,,,,,,,
Drifting longlines,2,2,2,2,2,2,0
IGNORE,6,6,6,6,0,6,0
Pole and line,12,12,12,12,10,12,0
Pots and traps,16,16,16,16,4,16,0
Purse seines,14,14,14,14,4,14,0
Set gillnets,6,6,6,6,0,6,0
Set longlines,10,10,10,10,4,10,0
Trawlers,29,29,29,29,5,29,0
Trollers,4,4,4,4,0,4,0


In [25]:
mapping_gearoverride = {
   'Drifting longlines' : 'Drifting_longlines',
    'IGNORE' : 'unknown_fishing',
    'Pole and line' : 'Pole_and_line',
    'Purse seines' : 'Purse_seines',
    'Set gillnets': 'Set_gillnets',
    'Set longlines': 'Set_longlines',
    'Trawlers' : 'Trawlers',
    'Trollers' : 'Trollers',
    'Pots and traps' : 'Pots_and_traps'
}

In [26]:
def decide_eu_geartype(mmsi, gear, subgear):
    gear = eu_gear2[gear]
    subgear = eu_gear2[subgear]
    if mmsi in mapping_gearoverride:
        label = mapping_gearoverride[mmsi]
    else:
        if gear != subgear and subgear is not None:
            label = gear+"|"+subgear
        else:
            label = gear
    return label

In [27]:
eu.head()

,mmsi,geartype,secondary_geartype,length,tonnage,power,num_mmsi
0,247306520,OTB,PS,18.20,32.0,199.0,1
1,247070960,OTB,PTM,23.02,122.0,478.0,1
2,250000887,PTM,OTM,50.70,1094.0,2500.0,1
3,219001588,SDN,GNS,18.50,57.2,221.0,1
4,247050110,OTB,PTM,17.80,31.0,128.7,1


In [28]:
eu['label'] = eu.apply(lambda row: decide_eu_geartype(row['mmsi'], row['geartype'], row['secondary_geartype']),
                       axis=1)


In [29]:
eu.groupby('label').count()

,mmsi,geartype,secondary_geartype,length,tonnage,power,num_mmsi
label,,,,,,,
Drifting_longlines,155,155,155,155,155,155,155
Drifting_longlines|Pots_and_traps,2,2,2,2,2,2,2
Drifting_longlines|Purse_seines,2,2,2,2,2,2,2
Drifting_longlines|Set_gillnets,15,15,15,15,15,15,15
Drifting_longlines|Set_longlines,30,30,30,30,30,30,30
Drifting_longlines|Trawlers,2,2,2,2,2,2,2
Pole_and_line,20,20,20,20,20,20,20
Pole_and_line|Drifting_longlines,5,5,5,5,5,5,5
Pole_and_line|Pots_and_traps,1,1,1,1,1,1,1


In [33]:
eu[['mmsi','label','length','tonnage','power','geartype','secondary_geartype']].to_csv('input_lists/eu_processed.csv')